In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from IPython.core.display import clear_output
from urllib.parse import quote

In [47]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Chrome()

In [3]:
errors = []

In [4]:
def expand_shadow_element(element):
    shadow_root = driver.execute_script('return arguments[0].shadowRoot',
                                        element)
    return shadow_root

In [5]:
def get_rt_movie_url(movie, year):
    clear_output(wait=True)

    url = r"https://www.rottentomatoes.com/search?search=" + quote(movie)

    driver.get(url)
    try:
        root1 = driver.find_element_by_tag_name('search-result-container')
        shadow_root1 = expand_shadow_element(root1)

        root2 = shadow_root1.find_element_by_css_selector(
            "search-result[type='movie']")
        shadow_root2 = expand_shadow_element(root2)

        html_of_interest = driver.execute_script(
            'return arguments[0].innerHTML', shadow_root2)
        sel_soup = BeautifulSoup(html_of_interest, 'html.parser')

        movie_url = sel_soup.find('media-row', {'releaseyear': year})
    except NoSuchElementException:
        errors.append(movie)
        print("Movie not found!!")
        return None

    #driver.quit()

    #print(type(movie_url))
    if movie_url is not None:
        print("get_rt_movie_url\nMovie:{},url:{}".format(
            movie, movie_url['url']))
        sleep(randint(1, 3))
        return movie_url['url']
    else:
        print("Movie Url not found!")
        sleep(randint(1, 3))
        return None

In [6]:
def get_rt_scores(movie, year):
    movie_url = get_rt_movie_url(movie, year)
    a = np.nan
    c = np.nan

    if movie_url is not None:
        try:
            response = get(movie_url)
            html_soup = BeautifulSoup(response.text, 'html.parser')

            x = html_soup.find('score-board')

            sleep(randint(1, 3))

            a = x['audiencescore']
            c = x['tomatometerscore']
        except TypeError:
            errors.append(movie)
            print("TypeError raised")

        print("get_rt_scores\nMovie:{},critic:{},audience:{}".format(
            movie, c, a))

    return [c, a]

In [7]:
imdb_df = pd.read_csv('imdb_meta.csv')

In [8]:
imdb_df

,movie,year,runtime,genre,imdb,meta,imdb_votes
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80,2360495
1,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84,2319761
2,Inception,2010,148 min,"Action, Adventure, Sci-Fi",8.8,74,2083710
3,Fight Club,1999,139 min,Drama,8.8,66,1868351
4,Pulp Fiction,1994,154 min,"Crime, Drama",8.9,94,1839615
...,...,...,...,...,...,...,...
8887,Falling Overnight,2011,86 min,"Drama, Romance",6.7,40,1002
8888,Fremde Haut,2005,97 min,Drama,7.0,60,1002
8889,Bam gua nat,2008,144 min,Drama,7.1,73,1001
8890,Awful Nice,2013,92 min,Comedy,4.9,42,1000


In [9]:
r=5

In [10]:
rt_df = imdb_df.copy()
test_df = rt_df.sample(r)

In [11]:
movies = imdb_df['movie']

In [12]:
years = imdb_df['year']

In [20]:
tomato = {}

In [ ]:
for i in range(15,200):
    a = get_rt_scores(movies[i], years[i])
    tomato[movies[i]] = a
    tom_df = make_db(tomato)

get_rt_movie_url
Movie:Batman Begins,url:https://rottentomatoes.com/m/batman_begins
get_rt_scores
Movie:Batman Begins,critic:84,audience:94


In [42]:
tomato

{'The Shawshank Redemption': ['91', '98'],
 'The Dark Knight': ['94', '94'],
 'Inception': ['87', '91'],
 'Fight Club': ['79', '96'],
 'Pulp Fiction': ['92', '96'],
 'Forrest Gump': ['71', '95'],
 'The Matrix': ['88', '85'],
 'The Lord of the Rings: The Fellowship of the Ring': ['91', '95'],
 'The Lord of the Rings: The Return of the King': ['93', '86'],
 'Interstellar': ['72', '86'],
 'The Dark Knight Rises': ['87', '90']}

In [43]:
def make_db(d):
    df = pd.DataFrame.from_dict(d, orient='index', columns=['rt_critic', 'rt_audience'])
    df.reset_index(inplace=True)
    df.rename(columns={'index':'movie'}, inplace=True)
    df.to_csv('rt.csv', index=False)
    return df

In [44]:
tom_df = make_db(tomato)

In [45]:
tom_df

,movie,rt_critic,rt_audience
0,The Shawshank Redemption,91,98
1,The Dark Knight,94,94
2,Inception,87,91
3,Fight Club,79,96
4,Pulp Fiction,92,96
5,Forrest Gump,71,95
6,The Matrix,88,85
7,The Lord of the Rings: The Fellowship of the Ring,91,95
8,The Lord of the Rings: The Return of the King,93,86
9,Interstellar,72,86


In [ ]:
t=1527

In [ ]:
m,y = movies[t], str(years[t])

In [ ]:
movies[t]

In [ ]:
print("{},{}".format(m,y))

In [ ]:
url = get_rt_movie_url(m,y)

In [ ]:
get_rt_scores(m, y)